<a href="https://colab.research.google.com/github/aayushrai/COVID-19-Mask-Recognition/blob/master/Model%20Training/Train_binary_keras_light_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!unzip "/content/drive/My Drive/dataset.zip"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Archive:  /content/drive/My Drive/dataset.zip
replace dataset/mask/with-mask-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

Using TensorFlow backend.


In [3]:
data = []
labels = []
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("dataset")))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (28, 28))
	image = img_to_array(image)
	data.append(image)
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	label = 1 if label == "mask" else 0
	labels.append(label)

In [7]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [4]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [5]:
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same",input_shape=(28,28,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))
# softmax classifier
model.add(Dense(2))
model.add(Activation("softmax"))

In [6]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [8]:
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/25
31/31 [==============================] - 1s 33ms/step - loss: 0.6807 - accuracy: 0.5897 - val_loss: 0.3416 - val_accuracy: 0.8783
Epoch 2/25
31/31 [==============================] - 1s 27ms/step - loss: 0.2684 - accuracy: 0.8896 - val_loss: 0.1427 - val_accuracy: 0.9466
Epoch 3/25
31/31 [==============================] - 1s 26ms/step - loss: 0.2208 - accuracy: 0.9202 - val_loss: 0.1388 - val_accuracy: 0.9525
Epoch 4/25
31/31 [==============================] - 1s 25ms/step - loss: 0.1909 - accuracy: 0.9274 - val_loss: 0.1121 - val_accuracy: 0.9525
Epoch 5/25
31/31 [==============================] - 1s 26ms/step - loss: 0.1628 - accuracy: 0.9284 - val_loss: 0.1556 - val_accuracy: 0.9466
Epoch 6/25
31/31 [==============================] - 1s 26ms/step - loss: 0.1783 - accuracy: 0.9233 - val_loss: 0.1112 - val_accuracy: 0.9614
Epoch 7/25
31/31 [==============================] - 1s 24ms/step - loss: 0.1350 - accuracy: 0.9489 - val_loss: 0.0736 - val_accuracy: 0.9733
Epoch 8/25
31

In [9]:
print("[INFO] saving mask detector model...")
model.save("light_mask_noMask.h5",include_optimizer=False)

[INFO] saving mask detector model...
